In [1]:
# test

In [1]:
#!/usr/bin/env python
# coding: utf-8

## All package and functions used are defined in utils
from utils import *
from calc_config import *

## Run the ipynb to process the result text file to retrieve/extract information for the calculation
import nbformat
from nbconvert import PythonExporter
from types import SimpleNamespace

# Convert the notebook to a Python script
notebook_filename = 'txt_read.ipynb'
exporter = PythonExporter()
source, metadata = exporter.from_filename(notebook_filename)

# Execute the Python script in a namespace
namespace = SimpleNamespace()
exec(source, namespace.__dict__)

## In the txt_read.ipynb, the following variables are defined.
num_orbitals = namespace.num_orbitals
num_particles = namespace.num_particles
num_spin_orbitals = namespace.num_spin_orbitals
obs_onebody_df = namespace.obs_onebody_df
obs_twobody_df = namespace.obs_twobody_df
abs_main = namespace.abs_main
pathfilename = namespace.pathfilename
nucleus_name = namespace.nucleus_name
num_spatial_orbitals = int(num_spin_orbitals/2)

### Define excitation list (to be used in the VQE)
# Define excitations list
def custom_excitation_list(num_spatial_orbitals: int,num_particles):
    # the function is of resemblance of the excitations selected in the Hamiltonian, but not really"
    non_repeat_list = list(combinations(range(0,sum(num_orbitals)),2))
    
    neut_orbitals_list = list(range(0,num_orbitals[0]))
    prot_orbitals_list = list(range(num_orbitals[0], sum(num_orbitals)))
    neut_state_list = list(range(0,num_particles[0]))
    prot_state_list = list(range(num_orbitals[0], num_orbitals[0] + num_particles[1]))

    init_state_indeces = neut_state_list + prot_state_list
    init_pair_list = HFground_pair_list(num_particles, num_orbitals)

    allowed_init = list(set(non_repeat_list) & set(init_pair_list))
    allowed_fina = non_repeat_list
    # allowed_fina = []
    # for fina in non_repeat_list:
    #     if not fina[0] or fina[1] in neut_state_list + prot_state_list:
    #         allowed_fina.append(fina)

    my_excitation_list = []

    for init in allowed_init:
        for fina in allowed_fina:
            if (fina[0] in init_state_indeces) or (fina[1] in init_state_indeces):
                pass
            elif ((init[0] in neut_orbitals_list) == (fina[0] in neut_orbitals_list) and
                (init[1] in neut_orbitals_list) == (fina[1] in neut_orbitals_list) and
                (init[0] in prot_orbitals_list) == (fina[0] in prot_orbitals_list) and
                (init[1] in prot_orbitals_list) == (fina[1] in prot_orbitals_list) 
               ):
                excitations = [init,fina];
                my_excitation_list.append(tuple(excitations) if (excitations not in my_excitation_list) else tuple())
    my_excitation_list.sort()
    return my_excitation_list
vqe_excitations = custom_excitation_list

## Setting up path and define names, pathfilename carry all the names for input and output
## This line was ran in the ipynb so, dont need to run once more 
# # # abs_main, nucleus, pathfilename = pathfilename_gen(pcname,input_txt)
os.chdir(abs_main)

# Record the start time for computation; And computation configuration.
start_time = datetime.now()

## The Hamiltonian
### Use the defined obs_onebody_df and obs_twobody_df to construct the Hamiltonian
from qiskit_nature.second_q.operators import FermionicOp
import pandas as pd
tmp_ham = {}

### One body Term: Single particle energy levels
for index, row in obs_onebody_df.iterrows():
    init_ = int(row['q_i']); fina_ = int(row['q_f'])
    the_onestring = "+_" +str(fina_) + " " + "-_" +str(init_)
    tmp_ham[the_onestring] = row['delta']

### Two body Terms: Pairing interaction
for index, row in obs_twobody_df.iterrows():
    init_1 = int(row['q_i1']); init_2 = int(row['q_i2']);
    fina_1 = int(row['q_f1']); fina_2 = int(row['q_f2']);
    the_twostring = "+_" +str(fina_1) + " " + "+_" +str(fina_2) + " " + "-_" +str(init_1) + " " + "-_" +str(init_2)
    tmp_ham[the_twostring] = row['V_ffii']

## The Hamiltonian Fermionic operator are given by
Hamiltonian = FermionicOp(tmp_ham, 
                          num_spin_orbitals=num_spin_orbitals, 
                          copy=False)


#### Setting up of the VQE algorithm
# Define a converter aka mapping method
from qiskit_nature.second_q.mappers import JordanWignerMapper, QubitConverter
qubit_converter = QubitConverter(JordanWignerMapper())

# from qiskit_nature.second_q.circuit.library.initial_states import HartreeFock
from qiskit_nature.second_q.circuit.library.ansatzes import UCC
from qiskit.algorithms.optimizers import ISRES,COBYLA,SLSQP
from qiskit.algorithms.minimum_eigensolvers import VQE, AdaptVQE
from qiskit import Aer

# Hamiltonian
Hamiltonian = qubit_converter.map(Hamiltonian)

## Define Estimator
from qiskit.primitives import Estimator
estimator = Estimator()

## Reference state / Initial state
from np_hartreefock import * ## Import custom init state
initial_state = HartreeFock(
    num_orbitals = num_orbitals,
    num_particles = num_particles,
    qubit_converter = qubit_converter)

## Ansatz
reps=1
var_form = UCC(
    num_particles=num_particles,
    num_spatial_orbitals=num_spatial_orbitals,
    excitations=vqe_excitations,
    qubit_converter=qubit_converter,
    reps=reps,
    initial_state=initial_state
)

# Define classical optimizer
optimizer=COBYLA(
    maxiter=optimizer_maxiter,
    disp=True, 
    tol = optimizer_tol)

# Define Solver
vqe = VQE(
    estimator = estimator,
    ansatz = var_form,
    optimizer = optimizer)

adapt_vqe = AdaptVQE(vqe,
                     threshold = 0.001,
                     max_iterations = 200)



In [21]:
var_form.decompose().decompose().decompose().decompose().decompose().decompose().count_ops()

OrderedDict([('u', 7908), ('cx', 7872)])

In [ ]:
Hamiltonian.depth()